In [ ]:
pip install ktrain==0.19.3

     |████████████████████████████████| 25.3MB 172kB/s 
     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 6.7MB 44.6MB/s 
     |████████████████████████████████| 983kB 44.8MB/s 
     |████████████████████████████████| 245kB 40.4MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 1.3MB 40.4MB/s 
     |████████████████████████████████| 471kB 38.5MB/s 
     |████████████████████████████████| 327kB 40.8MB/s 
     |████████████████████████████████| 450kB 43.0MB/s 
     |████████████████████████████████| 3.9MB 42.4MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 2.9MB 41.6MB/s 
     |████████████████████████████████| 890kB 42.8MB/s 
     |████████████████████████████████| 1.1MB 39.5MB/s 
  Created wheel for ktrain: filename=ktrain-0.19.3-cp36-none-any.whl size=25267700 sha256=ca2c86ce8d0cbcb917de9cc5dea57ce1fb1a454589d1feb5596709e255e08d01
  Stored 

In [ ]:
import ktrain
from ktrain import text
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
data = pd.read_csv("/content/function_assignment.csv")

In [ ]:
df = data.drop('function', axis = 1)
df = df.dropna()
df = df[df['jds'].str.len() > 20]
df.head(n=10)

In [ ]:
list_cat=[]
for i in df['job_title'].unique():
  l = len(df[df['job_title']==i])
  if l > 10:
    list_cat.append(i)

In [ ]:
df = df[df['job_title'].isin(list_cat)]

In [ ]:
x=[]
y=[]
for i in list(df['jds']):
    x.append(str(i))
for i in list(df['job_title']):
    y.append(str(i))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=42)

In [ ]:
txt=text.Transformer('distilbert-base-uncased', classes=list(df['job_title'].unique()))

In [ ]:
training=txt.preprocess_train(x_train,y_train)
validation=txt.preprocess_test(x_test,y_test)

In [ ]:
model=txt.get_classifier()
learner=ktrain.get_learner(model,train_data=training,val_data=validation,batch_size=2)

In [ ]:
learner.autofit(5e-5,15)

In [ ]:
predictor=ktrain.get_predictor(learner.model, txt)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
df2 = df.sample(200) 

In [ ]:
pred=[]
for i in list(df2['jds']):
  pred.append(predictor.predict(i))
df2['Pred'] = pred
print(accuracy_score(df2['job_title'],df2['Pred']))